# components

> Reusable ui components

In [ ]:
#| default_exp components

In [ ]:
#| hide
from nbdev.showdoc import *


# UI Components

> Reusable UI components for the contacts application

In [ ]:
#| default_exp components

In [ ]:
#| hide
from fasthtml.common import *
from fastcore.test import *
from todo.models import Contact
import IPython.display as display

## Page Layout

In [ ]:
#| export
def page_layout(content, title="Contacts App"):
    "Create a full page layout with header, content, and footer"
    return Titled(title,
        Header(
            H1("Contacts App"),
            Nav(
                A("All Contacts", href="/contacts", cls="nav-link"),
                A("Favorites", href="/contacts/favorites", cls="nav-link"),
                A("Add Contact", href="/contacts/new", cls="nav-link")
            ),
            Style("""
                header { 
                    background-color: #f5f5f5;
                    padding: 1rem;
                    margin-bottom: 2rem;
                }
                nav { display: flex; gap: 1rem; }
                .nav-link { text-decoration: none; }
                .htmx-indicator { opacity: 0; transition: opacity 200ms ease-in; }
                .htmx-request .htmx-indicator, .htmx-request.htmx-indicator { opacity: 1; }
                .contact-card { 
                    border: 1px solid #ddd; 
                    padding: 1rem; 
                    margin-bottom: 1rem;
                    border-radius: 4px;
                }
                .favorite { background-color: #fff8e1; }
                .search-bar {
                    margin-bottom: 2rem;
                    display: flex;
                    align-items: center;
                }
                .contact-form {
                    display: grid;
                    gap: 1rem;
                    max-width: 500px;
                }
            """)
        ),
        Main(content),
        Footer("© 2024 Contacts App")
    )

## Contact Card

In [ ]:
#| export
def contact_card(contact):
    "Create a card displaying contact information with action buttons"
    return Div(
        cls=f"contact-card {'favorite' if contact.favorite else ''}",
        id=f"contact-{contact.id}"
    )(
        H3(contact.name),
        P(f"Email: {contact.email}"),
        P(f"Phone: {contact.phone}"),
        P(f"Address: {contact.address}"),
        Div(
            Button("View", 
                hx_get=f"/contacts/{contact.id}",
                hx_target="#main-content",
                hx_push_url="true"),
            Button("Edit", 
                hx_get=f"/contacts/{contact.id}/edit",
                hx_target="#main-content",
                hx_push_url="true"),
            Button("Delete", 
                hx_delete=f"/contacts/{contact.id}",
                hx_confirm="Are you sure you want to delete this contact?",
                hx_target=f"#contact-{contact.id}",
                hx_swap="outerHTML")
        )
    )

Let's visualize a contact card with a sample contact:

In [ ]:
sample_contact = Contact(1, "John Doe", "john@example.com", "555-1234", "123 Main St", True)
card_html = str(contact_card(sample_contact))
display.HTML(card_html)

## Contact List

In [ ]:
#| export
def contact_list(contacts, search_query=""):
    "Create a list of contacts with search functionality"
    return Div(id="main-content")(
        Div(cls="search-bar")(
            Input(
                type="search", 
                name="q", 
                placeholder="Search contacts...",
                value=search_query,
                hx_get="/contacts",
                hx_trigger="keyup delay:300ms changed",
                hx_target="#contacts-list",
                hx_push_url="true",
                hx_indicator="#search-indicator"
            ),
            Img(
                id="search-indicator",
                cls="htmx-indicator",
                src="/static/img/spinner.svg",
                width="20",
                height="20"
            )
        ),
        Div(id="contacts-list")(
            *[contact_card(contact) for contact in contacts]
        ) if contacts else P("No contacts found.")
    )

## Contact Form

In [ ]:
#| export
def contact_form(contact=None, action="/contacts", method="post"):
    "Create a form for adding or editing contacts"
    return Div(id="main-content")(
        H2("Add Contact" if not contact else "Edit Contact"),
        Form(
            cls="contact-form",
            hx_post=action if not contact else f"/contacts/{contact.id}",
            hx_target="#main-content",
            hx_push_url="true"
        )(
            Label("Name", Input(name="name", value=contact.name if contact else "")),
            Label("Email", Input(name="email", type="email", value=contact.email if contact else "")),
            Label("Phone", Input(name="phone", value=contact.phone if contact else "")),
            Label("Address", Input(name="address", value=contact.address if contact else "")),
            Label(
                Span("Favorite"),
                Input(
                    name="favorite", 
                    type="checkbox", 
                    checked=contact and contact.favorite
                )
            ),
            Button("Save", type="submit"),
            Button(
                "Cancel", 
                type="button", 
                hx_get="/contacts",
                hx_target="#main-content",
                hx_push_url="true"
            )
        )
    )

## Contact Detail

In [ ]:
#| export
def contact_detail(contact):
    "Create a detailed view of a contact"
    return Div(id="main-content")(
        H2(contact.name),
        P(f"Email: {contact.email}"),
        P(f"Phone: {contact.phone}"),
        P(f"Address: {contact.address}"),
        P(f"Favorite: {'Yes' if contact.favorite else 'No'}"),
        Div(
            Button(
                "Edit", 
                hx_get=f"/contacts/{contact.id}/edit",
                hx_target="#main-content",
                hx_push_url="true"
            ),
            Button(
                "Back to List", 
                hx_get="/contacts",
                hx_target="#main-content",
                hx_push_url="true"
            )
        )
    )

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()